In [0]:
vardial2fasttext = {
    "ast":"ast",
    "cat":"ca",
    "fra":"fr",
    "ita":"it",
    "por":"pt",
    "spa":"es",
    "bak":"ba",
    "crh":"crh",
    "kaz":"kk",
    "kir":"ky",
    "tat":"tt",
    "tur":"tr"
}
fasttext2vardial = {ft_code:vd_code for vd_code,ft_code in vardial2fasttext.items()}




In [0]:
langs_to_process = {"roa" : ("ast", "cat", "fra", "ita", "por", "spa"), "trk":()}
for group_name, group_langs in langs_to_process.items():

    !wget -O train-{group_name}-uncovered -q https://raw.githubusercontent.com/ftyers/vardial-shared-task/master/train/{group_name}-uncovered
    !wget -O dev-{group_name}-uncovered -q https://raw.githubusercontent.com/ftyers/vardial-shared-task/master/dev/{group_name}-uncovered
    !wget -O dev-{group_name}-covered -q https://raw.githubusercontent.com/ftyers/vardial-shared-task/master/dev/{group_name}-covered

    !cat  train-{group_name}-uncovered >> {group_name}-alldata
    !cat  dev-{group_name}-uncovered >> {group_name}-alldata
    !cat  dev-{group_name}-covered >> {group_name}-alldata
    
    for lang_to_process in group_langs:
        lang_ft_code = vardial2fasttext[lang_to_process]
        !wget -q https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.{lang_ft_code}.vec
        
        
        alphabet = !cut -f2 {group_name}-alldata | grep -o . | sort -u
        for idx in range(len(alphabet)):
            elem = alphabet[idx]
            if elem in ['-','^',']']:
                alphabet[idx] = '\\'+elem
        alphabet = "".join(c for c in alphabet)
        
        vec_words_fn = f"{lang_ft_code}_vec_short"
        tracked_words_fn = f"{lang_to_process}_tracked"
        
        get_ipython().system(f"grep -P \"^{lang_to_process}\" {group_name}-alldata | cut -f2 | sort -u > {tracked_words_fn}")
        
        cmd_to_exec = f"grep -P \"^[{alphabet}]+\s\" wiki.{lang_ft_code}.vec | " + "grep -v -P \"^\w*\d\w*\" | cut -d$\' \' -f1 > " + f"{vec_words_fn}"
        get_ipython().system(cmd_to_exec)
        
        words_having_vectors_fn = f"{lang_to_process}_vectorized_words"
        words_non_having_vectors_fn = f"{lang_to_process}_nonvectorized_words"
        
        with open(tracked_words_fn, 'r', encoding='utf-8') as f_tracked,\
            open(vec_words_fn, 'r', encoding='utf-8') as f_vectors,\
            open(words_having_vectors_fn, 'w', encoding='utf-8') as f_vectorized,\
            open(words_non_having_vectors_fn, 'w', encoding='utf-8') as f_non_vectorized:
            
            track_words = set([line.strip() for line in f_tracked.readlines()])
            for line in f_vectors:
                line = line.strip()
                if line in track_words:
                    print(line, file=f_vectorized)
                else:
                    print(line, file=f_non_vectorized)
                    
        num_words_in_track = len(track_words)
        num_words_vectorized = !wc -l {words_having_vectors_fn} | cut -d$' ' -f1
        
        num_words_vectorized = int(num_words_vectorized[0])
        num_words_non_vectorized = num_words_in_track - num_words_vectorized
        
        print(", ".join([f"lang: {lang_to_process:>3}",
              f"words in track: {num_words_in_track:>6}",
              f"vectorized words num: {num_words_vectorized:>6}",
              f"non vectorized words num: {num_words_non_vectorized:>6}"]))
        
        tracked_vectors_fn = f"{lang_to_process}_tracked.vec"
        with open(f"wiki.{lang_ft_code}.vec", 'r', encoding="utf-8") as vectors_f, \
            open(tracked_vectors_fn, 'w', encoding="utf-8") as tracked_vectors_f:
                for line in vectors_f:
                    if line.strip().split()[0] in track_words:
                        print(line.strip(), file=tracked_vectors_f)
                        
            

lang: ast, words in track:   1000, vectorized words num:   1000, non vectorized words num:      0
lang: cat, words in track:  10000, vectorized words num:   9973, non vectorized words num:     27
lang: fra, words in track:   9986, vectorized words num:   9912, non vectorized words num:     74
lang: ita, words in track:   9998, vectorized words num:   9988, non vectorized words num:     10
lang: por, words in track:   9999, vectorized words num:   9974, non vectorized words num:     25
lang: spa, words in track:   9999, vectorized words num:   9999, non vectorized words num:      0


In [0]:
!ls

ast_nonvectorized_words  fra_nonvectorized_words  roa-alldata
ast_tracked		 fra_tracked		  sample_data
ast_tracked.vec		 fra_tracked.vec	  spa_nonvectorized_words
ast_vec_short		 fra_vectorized_words	  spa_tracked
ast_vectorized_words	 fr_vec_short		  spa_tracked.vec
cat_nonvectorized_words  ita_nonvectorized_words  spa_vectorized_words
cat_tracked		 ita_tracked		  train-roa-uncovered
cat_tracked.vec		 ita_tracked.vec	  train-trk-uncovered
cat_vectorized_words	 ita_vectorized_words	  trk-alldata
ca_vec_short		 it_vec_short		  wiki.ast.vec
dev-roa-covered		 por_nonvectorized_words  wiki.ca.vec
dev-roa-uncovered	 por_tracked		  wiki.es.vec
dev-trk-covered		 por_tracked.vec	  wiki.fr.vec
dev-trk-uncovered	 por_vectorized_words	  wiki.it.vec
es_vec_short		 pt_vec_short		  wiki.pt.vec
